In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [3]:
data = pd.read_csv('../input/train.csv')
data.head()

test_data_path = '../input/test.csv'
test_data = pd.read_csv(test_data_path)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import model_selection, preprocessing 

data.dropna(axis=0, subset=['Survived'], inplace=True)
y = data.Survived #np.log1p(data.SalePrice)

#data["trainOrTest"] = "train"
#test_data["trainOrTest"] = "test"
num_train = len(data)

df_all = pd.concat([data, test_data])
df_all = df_all.drop(['Survived','PassengerId','Name','Ticket','Cabin'], axis=1) #.select_dtypes(exclude=['object'])

for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_all[c].values)) 
        df_all[c] = lbl.transform(list(df_all[c].values))
        


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [5]:
#X = pd.get_dummies(X)
X = df_all[:num_train]
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25,
                                    random_state=1)

test_X = df_all[num_train:]

my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
val_X = my_imputer.transform(val_X)
test_X = my_imputer.transform(test_X)


#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#train_X = sc.fit_transform(train_X)
#val_X = sc.transform(val_X)
#test_X = sc.transform(test_X)
#print(train_X)

In [6]:
feature_names = [i for i in X.columns]
print(feature_names)
X.describe()

['Age', 'Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp']


,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp
count,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,1.538721,32.204208,0.381594,2.308642,0.647587,0.523008
std,14.526497,0.794231,49.693429,0.806057,0.836071,0.477990,1.102743
min,0.420000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,20.125000,1.000000,7.910400,0.000000,2.000000,0.000000,0.000000
50%,28.000000,2.000000,14.454200,0.000000,3.000000,1.000000,0.000000
75%,38.000000,2.000000,31.000000,0.000000,3.000000,1.000000,1.000000
max,80.000000,3.000000,512.329200,6.000000,3.000000,1.000000,8.000000


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestClassifier(random_state=1, n_estimators=100)
forest_model.fit(train_X, train_y)
preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, preds))

0.23766816143497757


In [8]:
from xgboost import XGBClassifier

my_model = XGBClassifier(n_estimators=100, learning_rate=0.05,
                         random_state=1,max_depth=7,
                         objective='binary:logistic')
my_model.fit(train_X, train_y, early_stopping_rounds=500, 
             eval_set=[(val_X, val_y)], eval_metric='auc',
             verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
#predictions = my_model.predict(train_X)
#y_pred = np.exp(predictions) - 1
#val_y_cor = np.exp(val_y)-1
#predictions[1:]
#from sklearn.metrics import log_loss
#print("Mean Absolute Error : " + str(log_loss(predictions, train_y)))
# max_depth=7,gamma=0,
 #                        colsample_bytree=1,

In [10]:
predictions = my_model.predict(val_X)
#from sklearn.metrics import mean_absolute_error
from sklearn.metrics import log_loss
print("Mean Absolute Error : " + str(log_loss(predictions, val_y)))
#print("Mean Absolute Error : " + str(mean_absolute_error(y_pred, val_y_cor)))

Mean Absolute Error : 6.660061756485901


In [11]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = feature_names)

Weight,Feature
0.2242 ± 0.0664,Sex
0.0726 ± 0.0200,Pclass
0.0323 ± 0.0174,Age
0.0314 ± 0.0150,Fare
0.0251 ± 0.0044,SibSp
0.0161 ± 0.0217,Embarked
0.0009 ± 0.0036,Parch


In [12]:
#from matplotlib import pyplot as plt
#from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
#pdp_goals = pdp.pdp_isolate(model=my_model, dataset=data, model_features=feature_names, feature='Fare')

# plot it
#pdp.pdp_plot(pdp_goals, 'Fare')
#plt.show()

In [ ]:
#test_data_path = '../input/test.csv'
#test_data = pd.read_csv(test_data_path)
#test_data.head()
#cols = list(X.columns.values)

In [ ]:
#test_X = test_data[X.columns]
#test_X = my_imputer.transform(test_X)
#test_X = sc.transform(test_X)

In [13]:
test_predictions = my_model.predict(test_X)
#test_y = np.exp(test_predictions) - 1
output = pd.DataFrame({'PassengerId': test_data.PassengerId,
                       'Survived': test_predictions})

output.to_csv('gender_submission.csv', index=False)